In [1]:
from numpy import array
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
macau = pd.read_csv('Macau-2022.csv', index_col='date', parse_dates = ['date'])

In [4]:
macau.head()

,arrival
date,
2011-01-01,2076066
2011-02-01,2164249
2011-03-01,2190865
2011-04-01,2338449
2011-05-01,2295551


In [5]:
mc_values = macau.values

In [6]:
n_steps_in = 4
n_steps_out = 2

In [7]:
X, y = split_sequence(mc_values, n_steps_in, n_steps_out)

In [8]:
n_features = 1

In [9]:
X = X.reshape(X.shape[0],X.shape[1], n_features)

In [10]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [11]:
model.fit(X, y, epochs=50, verbose=2)

Epoch 1/50
5/5 - 2s - loss: 5612328976384.0000 - 2s/epoch - 495ms/step
Epoch 2/50
5/5 - 0s - loss: 5245759913984.0000 - 43ms/epoch - 9ms/step
Epoch 3/50
5/5 - 0s - loss: 5090226208768.0000 - 36ms/epoch - 7ms/step
Epoch 4/50
5/5 - 0s - loss: 4862491754496.0000 - 36ms/epoch - 7ms/step
Epoch 5/50
5/5 - 0s - loss: 4569841008640.0000 - 35ms/epoch - 7ms/step
Epoch 6/50
5/5 - 0s - loss: 4116649869312.0000 - 39ms/epoch - 8ms/step
Epoch 7/50
5/5 - 0s - loss: 3300340269056.0000 - 34ms/epoch - 7ms/step
Epoch 8/50
5/5 - 0s - loss: 2483645841408.0000 - 35ms/epoch - 7ms/step
Epoch 9/50
5/5 - 0s - loss: 1548981501952.0000 - 36ms/epoch - 7ms/step
Epoch 10/50
5/5 - 0s - loss: 585047867392.0000 - 36ms/epoch - 7ms/step
Epoch 11/50
5/5 - 0s - loss: 238608826368.0000 - 35ms/epoch - 7ms/step
Epoch 12/50
5/5 - 0s - loss: 393820340224.0000 - 46ms/epoch - 9ms/step
Epoch 13/50
5/5 - 0s - loss: 266354425856.0000 - 30ms/epoch - 6ms/step
Epoch 14/50
5/5 - 0s - loss: 203993858048.0000 - 30ms/epoch - 6ms/step
Epoch 

In [12]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [13]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

537107
501758
 0.46 


In [14]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [15]:
model.fit(X, y, epochs=50, verbose=2)

Epoch 1/50
5/5 - 2s - loss: 5204535148544.0000 - 2s/epoch - 493ms/step
Epoch 2/50
5/5 - 0s - loss: 4808072757248.0000 - 24ms/epoch - 5ms/step
Epoch 3/50
5/5 - 0s - loss: 4407045390336.0000 - 23ms/epoch - 5ms/step
Epoch 4/50
5/5 - 0s - loss: 3979411456000.0000 - 21ms/epoch - 4ms/step
Epoch 5/50
5/5 - 0s - loss: 3477802319872.0000 - 22ms/epoch - 4ms/step
Epoch 6/50
5/5 - 0s - loss: 2948889051136.0000 - 23ms/epoch - 5ms/step
Epoch 7/50
5/5 - 0s - loss: 2473846636544.0000 - 22ms/epoch - 4ms/step
Epoch 8/50
5/5 - 0s - loss: 1949384966144.0000 - 22ms/epoch - 4ms/step
Epoch 9/50
5/5 - 0s - loss: 1412255055872.0000 - 22ms/epoch - 4ms/step
Epoch 10/50
5/5 - 0s - loss: 956900900864.0000 - 21ms/epoch - 4ms/step
Epoch 11/50
5/5 - 0s - loss: 543404720128.0000 - 22ms/epoch - 4ms/step
Epoch 12/50
5/5 - 0s - loss: 293879218176.0000 - 22ms/epoch - 4ms/step
Epoch 13/50
5/5 - 0s - loss: 195155689472.0000 - 22ms/epoch - 4ms/step
Epoch 14/50
5/5 - 0s - loss: 194895659008.0000 - 22ms/epoch - 4ms/step
Epoch 

In [16]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [17]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

720695
708934
 0.12 


In [29]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [30]:
model.fit(X, y, epochs=50, verbose=0)

In [31]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [33]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

628943
589350
 0.20 


In [18]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [19]:
y = y.reshape((y.shape[0], y.shape[1], n_features))

In [20]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

In [21]:
model.fit(X, y, epochs=50, verbose=2)

Epoch 1/50
5/5 - 2s - loss: 5755553972224.0000 - 2s/epoch - 494ms/step
Epoch 2/50
5/5 - 0s - loss: 5470566219776.0000 - 31ms/epoch - 6ms/step
Epoch 3/50
5/5 - 0s - loss: 5118296588288.0000 - 35ms/epoch - 7ms/step
Epoch 4/50
5/5 - 0s - loss: 4762870218752.0000 - 36ms/epoch - 7ms/step
Epoch 5/50
5/5 - 0s - loss: 4235411062784.0000 - 35ms/epoch - 7ms/step
Epoch 6/50
5/5 - 0s - loss: 3441681760256.0000 - 32ms/epoch - 6ms/step
Epoch 7/50
5/5 - 0s - loss: 2839641849856.0000 - 32ms/epoch - 6ms/step
Epoch 8/50
5/5 - 0s - loss: 2021674582016.0000 - 32ms/epoch - 6ms/step
Epoch 9/50
5/5 - 0s - loss: 1341266722816.0000 - 34ms/epoch - 7ms/step
Epoch 10/50
5/5 - 0s - loss: 985738575872.0000 - 35ms/epoch - 7ms/step
Epoch 11/50
5/5 - 0s - loss: 857732677632.0000 - 35ms/epoch - 7ms/step
Epoch 12/50
5/5 - 0s - loss: 740645339136.0000 - 29ms/epoch - 6ms/step
Epoch 13/50
5/5 - 0s - loss: 666792296448.0000 - 30ms/epoch - 6ms/step
Epoch 14/50
5/5 - 0s - loss: 557970489344.0000 - 32ms/epoch - 6ms/step
Epoch 

In [22]:
x_input = array([328245, 801300, 820870, 694430])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [23]:
for i in range(len(yhat[0])):
    print(int(yhat[0][i]))
MAPE = ((abs(694430 - yhat[0][0])/694430) + (abs(655505 - yhat[0][1])/655505))
print(" %.2f " %(MAPE))

610480
645019
 0.14 
